# Run a tier sequence

This tutorial explains the core functionality of the `bonsai_ipcc` package.
Let´s assume we want to determine the CO2 emissions from waste incineration based on the tier 1 approach. According to the IPCC guidelines, different waste fractions are available. Here we chose year 2010, Germany and waste type `msw_plastics`, which stands for municipal plastic waste (you can check all availbe waste types by `my_ipcc.waste.incineration.dimension.waste_type`) and unspecified incineration technology.

In [1]:
import bonsai_ipcc
my_ipcc = bonsai_ipcc.IPCC()
my_ipcc.waste.incineration.sequence.tier1_co2(year=2010, region="DE",wastetype="msw_plastics", incintype="inc_unspecified", uncertainty="def")

2023-12-19 14:39:42,218 - INFO - Incineration sequence started --->
/Users/TN76JP/Documents/coderefinery/ipcc/ipcc/src/ipcc/_sequence.py:87: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  new_c = conc_df.loc[c][j]


KeyError: "Coordinate '(2010, 'DE')' or its concordance not found for parameter 'urb_population' with table 'urb_population'. If uncertainty analysis, also check required properties (e.g. max, min)."

Trying to do so, generates a an KeyError mentioning that it could not find the coordinate for year 2010 and Germany in the paramter table `urb_population`. Let´s have a look into this parameter table.

In [2]:
my_ipcc.waste.incineration.parameter.urb_population

,,,value,unit
year,region,property,,
0,dummy,dummy,0,cap


It is empty. The reason is that the IPCC guidelines do not provide default data for all parameters. In these cases we need to add the data to the specific parameter.
Let´s do this for the urban population in Germany in 2010.

In [3]:
import pandas as pd

# urban population
d = {
    "year": [2010,2010,2010,2010,2010],
    "region": ["DE","DE","DE","DE","DE"],
    "property": [
        "def","min","max","abs_min","abs_max"
    ],
    "value": [
        62940432,61996325.52,63884538.48,0.0,"inf",
    ],
    "unit": [
    "cap/yr","cap/yr","cap/yr","cap/yr","cap/yr",
    ],
}
urb_pop = pd.DataFrame(d).set_index(["year", "region", "property"])

my_ipcc.waste.incineration.parameter.urb_population=urb_pop

And run the sequence again.

In [4]:
my_tier = my_ipcc.waste.incineration.sequence.tier1_co2(
          year=2010, region="DE",wastetype="msw_plastics", incintype="inc_unspecified", uncertainty="def")
my_tier.__dict__

2023-12-19 14:39:57,025 - INFO - Incineration sequence started --->
/Users/TN76JP/Documents/coderefinery/ipcc/ipcc/src/ipcc/_sequence.py:87: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  new_c = conc_df.loc[c][j]
2023-12-19 14:39:57,029 - INFO - 'Coordinates (2010, 'DE')' has been replaced by '[2006, 'DE']' during reading parameter table 'msw_gen_rate'
2023-12-19 14:39:57,031 - INFO - 'Coordinates (2010, 'DE')' has been replaced by '[2006, 'DE']' during reading parameter table 'msw_frac_to_incin'
2023-12-19 14:39:57,034 - INFO - 'Coordinates (2010, 'DE', 'msw_plastics')' has been replaced by '[2006, 'DE', 'msw_plastics']' during reading parameter table 'msw_type_frac'
2023-12-19 14:39:57,036 - INFO - 'Coordinates (2010, 'DE', 'inc_unspecified')' has been replaced by '[2006, 'World', 'inc_unspecified']

{'signature': {'year': 2010,
  'region': 'DE',
  'wastetype': 'msw_plastics',
  'incintype': 'inc_unspecified',
  'uncertainty': 'def'},
 'urb_population': Step(position=1, year=2010, unit='cap/yr', value=62940432, type='data'),
 'msw_gen_rate': Step(position=2, year=2006, unit='t/cap/yr', value=0.6, type='data'),
 'msw_frac_to_incin': Step(position=3, year=2006, unit='kg/kg', value=0.37, type='data'),
 'msw_type_frac': Step(position=4, year=2006, unit='kg/kg', value=0.104, type='data'),
 'sw_per_treat': Step(position=5, year=2010, unit='Gg/year', value=1453.1686940159996, type='elementary'),
 'incintype_frac': Step(position=6, year=2006, unit='kg/kg', value=1, type='data'),
 'sw_per_tech': Step(position=7, year=2010, unit='Gg/year', value=1453.1686940159996, type='elementary'),
 'dm': Step(position=8, year=2006, unit='kg/kg', value=1.0, type='data'),
 'cf': Step(position=9, year=2006, unit='kg/kg', value=0.75, type='data'),
 'fcf': Step(position=10, year=None, unit='kg/kg', value=1.0,

Each step involved in the calculation is stored in the dictionary. It starts with the input signature and ends with the final result of CO2 emsissions.